In [2]:
from jupyter_plotly_dash import JupyterDash
import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State
import base64
import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
from ProjectOne import AnimalShelter

###########################
# Data Manipulation / Model
#########################
# FIX ME change for your username and password and CRUD Python module name
username = "aacuser"
password = "kash2274"
shelter = AnimalShelter(username, password)

filters = {
    "WR": {
        "breed":{
            "$in":[
                "Labrador Retriever Mix",
                "Chesapeake Bay Retriever",
                "Newfoundland"
            ]
        },
        "sex_upon_outcome":"Intact Female",
        "age_upon_outcome_in_weeks":{
            "$gte":26,
            "$lte":156,
        }
        
    },
    "MW": {
        "breed":{
            "$in":[
                "German Shepherd",
                "Alaskan Malamute",
                "Old English Sheepdog",
                "Siberian Husky",
                "Rottweiler"
            ]
        },
        "sex_upon_outcome":"Intact Male",
        "age_upon_outcome_in_weeks":{
            "$gte":26,
            "$lte":156,
        }
        
    },
    "DIT": {
        "breed":{
            "$in":[
                "Doberman Pinscher",
                "German Shepherd",
                "Golden retriever",
                "Bloodhound",
                "Rottweiler"
            ]
        },
        "sex_upon_outcome":"Intact Male",
        "age_upon_outcome_in_weeks":{
            "$gte":20,
            "$lte":300,
        }
    },
    "R":{
        
    }
    
}

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')


image_filename = 'GraziosoSalavare.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center([html.B(html.H1('SNHU CS-340 Dashboard')), html.Span('Sean Aceituno')]),
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), width=100),
    html.Hr(),
    html.Div(
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'WR' },
                {'label': 'Mountion or Wilderness', 'value': 'MW'},
                {'label': 'Disaster or Individual Tracking', 'value': 'DIT'},
                {'label': 'Reset', 'value': 'R'}],
            value='DIT',
            labelStyle={'display': 'inline-block'}
        )          
    ),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[],
        data=[],
        page_size=10,
        row_selectable="single",
        sort_action="native",
       
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type): 
    df = pd.DataFrame.from_records(shelter.read(filters[filter_type]))
    columns = [c for c in df.columns if c != "_id"]
    cols=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in columns]
    data=[{key: r[key] for key in columns} for r in df.to_dict('record')]      
    return (data,cols)

@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "data")])
def update_graphs(viewData):
    breedCounts = {}
    for row in viewData:
        if row["breed"] in breedCounts:
            breedCounts[row["breed"]] += 1
        else:
            breedCounts[row["breed"]] = 1
            
        names, values = zip(*breedCounts.items())
    return [
        dcc.Graph(       
            figure=px.pie(viewData, names=names, values=values),
            style={'width': '500px', 'height': '400px'}
        )    
    ]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "data")])
def update_map(viewData):
    layers=[dl.TileLayer()]
    for row in viewData:
        layers.append(dl.Marker(position=(row["location_lat"], row["location_long"]),
                               children=[dl.Tooltip("name: {name} breed: {breed}".format_map(row))]))
    return [
        dl.Map(style={'width': '500px', 'height': '400px'}, children=layers)
    ]


app